Name : Nagashekar Ananda

Department : CSE

Course : 2212-INSY-5336-003

Final Project 2.0

1.(100 points) Write a python program that fetches movie information for the top 500 most popular movies from Metacritics. On this websites, there is an option to show the top movies. 
On Metacritics, it is called “Movies of All Time”

You will first write python script that collect the movie information for the top 500 movies from each website and store them in a comma separated file (called [your name]_movies.csv). 

In addition to the csv file, the data should also be stored in a SQLite database called MovieInfoDatabase in the directory that your Jupyter Notebook code will be executed from. The MovieInfoDatabase should have a table called MovieInfoTable. 

Next, from the movie information you have collected, extract 2 pieces of information: The director, and the cast (actors/actresses). Build a dictionary of the movies that contain these information. Arrange them in any way you prefer but make sure we can access the information we need at any time. 


############################################################################################################################

Web Scrapping Approach - 

1.Visit https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page= for 5 times to get top 500 movie's links to to specific movie pages by getting href from specific parts of the HTML results.

2.Iterate through each link and collect each movie name and movie details link to retreieve other information about the movie.

3.Scrap the directors, cast, principle cast lists from the details page and store it.

4.Coverting each piece of information of each movie into a whole and stroing it .csv, in sqllite3 database and also locally.

5.Take interactive inputs from the user and select appropriate user sub category fetch it and display it.


Scrapped Data Anaylization -

1.Director Analysis - to calculate a list of directors with number of appearences in the top 500 movies at Metacritic website greater than the threshold.

2.Actor Analysis - to calculate a list of Actors with number of appearences in the top 500 movies at Metacritic website greater than the threshold.

3.Cosine Similarity between two Directors - 
  combination of all the cast that director 1 and 2 have worked with and individual count of cast member appearences with the directors as a base cosine similarity is calculated using the formaula which ranges from 0-1 indicating the similarity in cast with which both directors have worked.
  
4.Cosine Similarity between 5 actors among them as described in step 3 and to find the most popular supporting actor/actress to these actors.


############################################################################################################################

***Run Block Instructions before every Block and Run the Program***

Run Next 2 Cells

In [1]:
from bs4 import BeautifulSoup                                      #to import all the functionalities offered by beautiful soup
import ssl                                                         # connection establishment
import requests                                                    # to make request to web addresses
import re                                                          # for regular expression comparisons
import csv                                                         # to be able to write to a csv file
import sqlite3                                                     # to be able to create and manipulate table entries


# Ignore SSL certificate errors
ctx = ssl.create_default_context()                                 # creating default connection context
ctx.check_hostname = False                                         # setting check_hostname to false
ctx.verify_mode = ssl.CERT_NONE                                    # setting verify_mode to None ssl certificate

#connection with sqlite DB
conn = sqlite3.connect('MovieInfoDatabase.db')                     # connecting to sqllit db 
c = conn.cursor()                                                  # getting cursor object to traverse the db table

dropTableStatement = "DROP TABLE IF EXISTS MovieInfoTable"                   # dropping table if it exists

c.execute(dropTableStatement)                                      # executing above SQL statement

c.execute("""CREATE TABLE IF NOT EXISTS MovieInfoTable(            
                MOVIE_NAME text, 
                DIRECTORS_LIST text,
                PRINCIPLE_CAST_LIST text,
                CAST LIST text )""")                                # creating table with specific fields

conn.commit()                                                       # committing changes to db


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}         # header to make it seem like we are accessing the website from a browser and we are not a bot    

***Run the Next Block and wait till the completion Message and then proceed***

In [3]:

all_movies = []                                                       # array to store all movie information scrapped from website

for i in range(5):                                                                                            # to scrap 5 pages to get 500 movies
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)    # creating specific url to access the 5 pages
    page = requests.get(url, headers = headers)                                                               # making get request to the specific url
    soup = BeautifulSoup(page.content, "html.parser")                                                         # scrapping page contecnt and parsing it into html using beautifulsoup
    hundred = soup.find_all('td', class_ = 'clamp-summary-wrap')                                              # we are finding td in all the html data scrapped whose class attribute is class-summary-wrap
    all_movies.extend(hundred)                                                                                # adding all the scrapped data's specific td data into all movies array

all_movie_links = []                    # creating all_movie_links list
all_movie_names = []                    # creating all_movie_names list
 
for movie in all_movies:                                                    # looping through each movie in allmovies list
    base_url = 'https://www.metacritic.com'                                 # base url string 

    movie_name_find = movie.find('h3')                                      # trying to find h3 tag in movie data
    all_movie_names+= movie_name_find.contents                              # adding the movie name from the contents of the h3 to all_movie_names list

    movie_link_tag = movie.find('a', class_ = 'title')                      # trying to find a tag with class attr title                     
    movie_link = movie_link_tag.get('href', None)                           # getting the href inside it 

    movie_link = base_url + movie_link                                      # adding that url to base url
    detail_page = requests.get(movie_link, headers = headers)               # making get request with above creating movie link
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")         # scraping entire data from the page and parsing it into html
    div = detail_soup('div', class_= 'see_all pad_btm1')                    # trying to find all div with attr see_all and pad_html
    for line in div:                                                        # iterating through above found data
        hyperlink = line.find('a')                                          # trying to find a tag
        detail_link = hyperlink.get('href', None)                           # getting href from a tag                 
        detail_link = 'https://www.metacritic.com'+ detail_link             # forming detailed link
        all_movie_links.append(detail_link)                                 # appending to the all_movie_links list

all_movies_directors = []                                                   # creating empty all_movie_directors list
all_movies_principle_cast = []                                              # creating empty all_movie_principle_cast list
all_movies_cast  = []                                                       # creating empty all_movie_cast list
all_movie_principle_cast_role = []                                          # creating empty all_movie_principle_cast_role list 
all_movies_cast_role  = []                                                  # creating empty all_movie_cast_role list

for i,movie_link in enumerate(all_movie_links):                                                 # iterating through all the movie links
    complete_details_page = requests.get(movie_link, headers = headers)                         # making get request on each movie link
    complete_details_soup = BeautifulSoup(complete_details_page.content, "html.parser")         # getting all contents of the file

    director_table = complete_details_soup.select('table[summary*="Director Credits"]')         # selecting specific table with summary attr consisting of director credits
    if len(director_table)>0:                                                                   # checking if the data is present or not
        director_line = director_table[0].find_all('td', class_ = 'person')                     # checking if specific td element with class attr person is present and finding it
        directors = list()                                                                      # creating empty directors list for single movie
        for line in director_line:                                                              # iterating through each value found
            hyperlink = line.find('a')                                                          # finding the a tag
            director = hyperlink.get_text()                                                     # getting the contents of the a tag
            director_name_str = str(director.strip()).lower()                                   # stripping the contents and converting it into lowercase
            directors.append(director_name_str)                                                 # adding to directors list
        all_movies_directors.append(directors)                                                  # adding final directors list for the movie to all directos list
    else:
        all_movies_directors.append([])                                                         # if no data was found appending empty list

    principle_cast_table = complete_details_soup.select('table[summary*="Principal Cast Credits"]')     # selecting specific table with summary attr consisting of principle cast
    if len(principle_cast_table)>0:                                                                     # checking if the data is present or not
        principle_cast_line = principle_cast_table[0].find_all('td', class_ = 'person')                 # checking if specific td element with class attr person is present and finding it
        principle_cast = list()                                                                         # creating empty principle castgle movie
        for line in principle_cast_line:                                                                 # iterating through each value found
            hyperlink = line.find('a')                                                                   # finding an a tag
            principle_cast_member = hyperlink.get_text()                                                 # getting contents of the tag
            principle_cast_name_str = str(principle_cast_member.strip()).lower()                         # stripping the principle cast member name and coverting to lowercase
            principle_cast.append(principle_cast_name_str)                                               # appeding each member to list
        all_movies_principle_cast.append(principle_cast)                                                 # appedning the list formed above to final all principle cast members
    else:
        all_movies_principle_cast.append([])                                                             # if no data then append empty list
    
    if len(principle_cast_table)>0:                                                                     # checking if the data is present or not
        principle_cast_line = principle_cast_table[0].find_all('td', class_ = 'role')                    # checking if specific td element with class attr role is present and finding it
        principle_cast_role = list()                                                                     # creating empty principle cast ROLE movie
        for line in principle_cast_line:                                                                 # iterating through each value found
            hyperlink = line.find('a')                                                                   # finding an a tag
            principle_cast_member_role = hyperlink.get_text()                                            # getting contents of the tag
            principle_cast_name_role_str = str(principle_cast_member_role.strip()).lower()               # stripping the principle cast member name and coverting to lowercase
            principle_cast_role.append(principle_cast_name_role_str)                                     # appeding each member to list
        all_movie_principle_cast_role.append(principle_cast_role)                                        # appedning the list formed above to final all principle cast members
    else:
        all_movie_principle_cast_role.append([])                                                         # if no data then append empty list
    
      
    def cast_only(cast_only):                                                                                        # function to check attribute value
        return cast_only is not None and cast_only[-12:]=="Cast Credits" and cast_only[-16:]!="pal Cast Credits"     # checcking if it not principle cast
    
    cast_table = complete_details_soup.find('table', class_ = 'credits', summary = cast_only )                       # trying to find table with attribute
    if cast_table!=None:                                                                                             # checking if the data found is not None
        cast_line = cast_table.find_all('td', class_ = 'person')                                                     # finding all the td's with class attr person
        cast = list()                                                                               #creating an empty list
        for line in cast_line:                                                                      # iterating through each line in data found
            hyperlink = line.find('a')                                                              # trying to find a tags 
            cast_member = hyperlink.get_text()                                                      # getting the contents of the a tag
            cast_name_str = str(cast_member.strip()).lower()                                        # stripping the contents and converting to lowercase
            cast.append(cast_name_str)                                                              # adding to list
        all_movies_cast.append(cast)                                                                # adding above formed list to total cast list
    else: 
        all_movies_cast.append([])                                                                  # if data is not found adding empty array
    
    if cast_table!=None:                                                                            # checking if the data found is not None
        cast_line = cast_table.find_all('td', class_ = 'role')                                      # finding all the td's with class attr role
        cast_role = list()                                                                          #creating an empty list
        for line in cast_line:                                                                      # iterating through each line in data found
            hyperlink = line.find('a')                                                              # trying to find a tags 
            cast_member_role = hyperlink.get_text()                                                 # getting the contents of the a tag
            cast_name_role_str = str(cast_member_role.strip()).lower()                              # stripping the contents and converting to lowercase
            cast_role.append(cast_name_role_str)                                                    # adding to list
        all_movies_cast_role.append(cast_role)                                                      # adding above formed list to total cast list
    else: 
        all_movies_cast_role.append([])                                                             # if data is not found adding empty array


print("All Movie Details Collected !!")                                                              # printing message that all movie data was collected

#writing to CSV
with open('Nagashekar_Ananda_movies.csv','w', newline="") as f:                                      # opening the CSV file if present or creating one
    writer = csv.writer(f)                                                                           # initializing a write to csv

    writer.writerow(['MOVIE NAME', 'DIRECTORS LIST', 'PRINCIPLE CAST LIST', 'CAST LIST'])            # writing the column names to csv

    for i in range(len(all_movie_links)):                                                            # itertating for every movie link
        listtowrite = []                                                                             # creating an empty list to write
        listtowrite.append(str(all_movie_names[i]).lower())                                          # appending each movie name
        listtowrite.append(",".join(all_movies_directors[i]))                                        # adding corresponding directors list
        listtowrite.append(",".join(all_movies_principle_cast[i]))                                   # adding corresponding principle cast list
        listtowrite.append(",".join(all_movies_cast[i]))                                             # adding corresponding cast list 
        writer.writerow(listtowrite)                                                                 # writing entire list produced onto csv

        #inserting into DB
        c.execute(" INSERT INTO MovieInfoTable VALUES (:movie_name,:directors_list,:principle_cast_list,:cast_list)",{ 'movie_name':listtowrite[0], 'directors_list': listtowrite[1], 'principle_cast_list' : listtowrite[2], 'cast_list': listtowrite[3]})    # inserting into db table

#db commit and close      
conn.commit()                                                                          # commiting db
conn.close()                                                                           # closing db connection

movies_dictionary = {}                                                                 # creating empty movies dictionary
movies_full_info_list = []


for i in range(len(all_movie_links)):                                                    #iterating through each movi link
    temp_dictionary = {}                                                                 # creating empty dictionary
    temp_dictionary['movie_name'] = str(all_movie_names[i]).lower()                      # adding key value pair of movie and moviename
    temp_dictionary['director'] = all_movies_directors[i]                                # adding key value pair of directorand director list
    temp_dictionary['principle_cast'] = all_movies_principle_cast[i]                     # adding key value pair of principle castand principle cast list
    temp_dictionary['cast'] = all_movies_cast[i]                                         # dding key value pair of cast and cast list

    movies_dictionary[str(all_movie_names[i]).lower()] = temp_dictionary                 # adding key value pair of movienamd and above contrusted dictionary
    movies_full_info_list.append(temp_dictionary)

print("All Movie Details Stored !!")                    # printing a message that all details are stored

                

All Movie Details Collected !!
All Movie Details Stored !!


***The above block might take more than 4 minutes to execute and return the results,***

**Completion is indicated with the messages -**

**All Movie Details Collected !!**
**All Movie Details Stored !!**   

############################################################################################################################

**Run all the above block before you start with this question**

In [5]:
def task1():                                                                               
    movie_input = input('Which movie do you want to check?').strip().lower()               # taking movie name input from user and stripping down and converting to lowercase
    while True:                                                                            #iterating infinite times
        try:                                                                               # try to execute block
            flag = 0                                                                       # flag variable initialised to 0
            if movie_input not in movies_dictionary.keys():                                # checking if movie is present in the dictionary
                flag = 1                                                                   # setting flag to 1 and raising exception
                raise
            sub_cat_input = input('What information about this movie do you want to check? (Choose director or cast or principle_cast)').strip().lower()  # taking input for info to check from user and coverting to lowercase
            if sub_cat_input not in ['director', 'cast', 'principle_cast']:                 # is the category does not exist
                flag = 2                                                                    # set flag to 2 and raise exception
                raise
            break                                                                           # else if it is present break
        except:                                                                             # if try does not execute completely
            if flag==1:                                                                                        # checking if the flag is set to 1
                movie_input = input('Please Valid Movie Name that you want to check?').strip().lower()         # asking user to enter valid movie name
            elif flag==2:                                                                                      # checking if fla is 2 
                sub_cat_input = input('Please Enter valid information about this movie you want to check? (Choose director or cast or principle_cast)').strip().lower()   #asking user to enter valid category name
        
    indices = []                                                                           # empty indices list
    for i,information in enumerate(movies_full_info_list):                                 # iterating through each movie information
        if information['movie_name'] == movie_input:                                       # checking if movie name is equal to user enetered value
            indices.append(i)                                                              # appending to indices list
    print("\nThere are",len(indices)," movie(s) with the name  - ",movie_input)            # priting statement on how many movies are found
    
    for i in indices:                                                                      # iterating through each indices found
        print("\n")
        if sub_cat_input == "director":                                                    # checking if sub category is director
            print("Movie Name - ",movie_input)                                              # printing movie name
            print("\nDirector(s) - ")                                           
            for d in movies_full_info_list[i]['director']:                                  # iterating and printing all directors associated with the movie
                print(d)

        elif sub_cat_input == "cast":                                                       # checking if sub category is cast                    
            print("Movie Name -",movie_input)                                                # printing movie name
            print("\nCast -")
            for c in movies_full_info_list[i]['cast']:                                      # iterating and printing all cast associated with the movie
                print(c)

        elif sub_cat_input == "principle_cast":                                               # checking if sub category is principle cast    
            print("Movie Name - ",movie_input)                                                 # printing movie name
            print("\nPrinciple Cast - ") 
            for pc in movies_full_info_list[i]['principle_cast']:                              # iterating and printing all principle cast associated with the movie
                print(pc)
    
task1()  

Which movie do you want to check?the godfather
What information about this movie do you want to check? (Choose director or cast or principle_cast)diRector

There are 1  movie(s) with the name  -  the godfather


Movie Name -  the godfather

Director(s) - 
francis ford coppola


**If you are not seeing any information even the movie is found it means that the movie's detail page does not consist of that data**

############################################################################################################################

1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?

**Run all the above block before you start with this question**

**Run the Next 4 Cells one by one to execute and check the results by entering appropriate inputs**

In [6]:
def director_analysis():                                                                          # function to analysis all the directors
    all_individual_directors_list = list()                                                        # empty individual directors list
    for sublist_directors in all_movies_directors:                                                # iterating through all movie directors list
        for single_director in sublist_directors:                                                 # iterating through each individual director
            all_individual_directors_list.append(single_director)                                 # appending each single director to the list

    director_appearences = {}                                                                     # empty dictionary to store director appearances

    for single_director in all_individual_directors_list:                                         # iterating through each direction in the top 500 movies
        director_appearences[single_director] = director_appearences.get(single_director,0) + 1   # adding a count to their key if found - getting their appearence count

    successful_directors = []                                                                     # empty successful directors list
    for single_director, movies_count in director_appearences.items():                            # iterating through each director 
        if movies_count>=4:                                                                       # checking if they have appearred more than 4 or equal
            successful_directors.append(single_director)                                          # adding the director name to successsful directors list

    print("Here are a few Successful Directors who have Directed 4 or more Movies - \n")          # printing message about successful directors
    for single_director in successful_directors:                                                  # iterating and print each director in the successful directors list
        print(single_director)
    return director_appearences                                                                   # also returning the director appearances

In [7]:
def actor_analysis():                                                                             # actor analysis function
    all_individual_actors_list = list()                                                           # empty list to store individual actors
    for sublist_actors in all_movies_principle_cast:                                              # each cast member and principle cast from each movie are added to above list
        for single_actor in sublist_actors:
            all_individual_actors_list.append(single_actor)
    for sublist_actors in all_movies_cast:
        for single_actor in sublist_actors:
            all_individual_actors_list.append(single_actor)

    actors_appearences = {}                                                                      # dictionary to store actor appearences

    for single_actor in all_individual_actors_list:                                              # iterating through each actor and getting their count in all the movies
        actors_appearences[single_actor] = actors_appearences.get(single_actor,0) + 1

    successful_actors = []                                                                       # empty successful actors list
    for single_actor, movies_count in actors_appearences.items():                               #iterating and adding each actor who had appeared more than 4 times in movies
        if movies_count>=4:
            successful_actors.append(single_actor)

    print("Here are a few Successful Actors/Actresses who have Acted 4 or more Movies(as Pricniple Cast/ General Cast) - \n")   # printing message regarding successful actors
    for single_actor in successful_actors:                                                         # printing all the actors with 4 or more movie appearrences
        print(single_actor) 
        
    return actors_appearences                                                                  # returning actor appearences count

In [8]:
director_appearences = director_analysis()

Here are a few Successful Directors who have Directed 4 or more Movies - 

orson welles
francis ford coppola
alfred hitchcock
michael curtiz
ben sharpsteen
bill roberts
wilfred jackson
john huston
stanley kubrick
george cukor
steve mcqueen
robert altman
john ford
martin scorsese
mike leigh
steven spielberg
peter jackson
spike lee
hirokazu koreeda
howard hawks


In [9]:
actors_appearences = actor_analysis()

Here are a few Successful Actors/Actresses who have Acted 4 or more Movies(as Pricniple Cast/ General Cast) - 

joseph cotten
orson welles
al pacino
marlon brando
james stewart
humphrey bogart
ethan hawke
cary grant
mahershala ali
walter catlett
ray collins
daniel day-lewis
george c. scott
karl malden
rudy bond
henry fonda
kyle chandler
lucas hedges
ward bond
brad pitt
garret dillahunt
paul giamatti
sarah paulson
john qualen
harvey keitel
robert de niro
tom hanks
chris pratt
cate blanchett
rooney mara
julie delpy
isabelle huppert
tom hardy
john wayne
lesley manville
timothy spall
ralph fiennes
kathryn newton
laurie metcalf
saoirse ronan
timoth
tracy letts
andy serkis
orlando bloom
adam driver
scarlett johansson
wallace shawn
emma stone
j.k. simmons
frances mcdormand
michael stuhlbarg
thomas mitchell
liev schreiber
michael keaton
mathieu amalric
willem dafoe
caleb landry jones
katharine hepburn
sterling hayden
george young
alec guinness
mark gatiss
ian holm
scott glenn
amy adams
forest 

***Looking at the number of successful actors/actresses and directors and comparing them with the total number of actors/actresse and directors respectively it is visible that only a small portion of them are successful and rest of them are still struggling to find good opportunities to showcase their talent***

###########################################################################################################################

2. Create a dictionary of actors/actresses that the directors have worked together with in each movie, then calculate their cosine similarity, which directors work with similar groups of actors/actresses? Use director name as the dictionary name, actor/actress name as the key, and the times they have worked together in a movie as the value. For example: Michael Bay = {‘ Bruce Willis’: 50, ‘Ben Affleck’:20, ‘Liam Neeson’:10}, Steven Spielberg = {‘Liam Neeson’: 30, ‘Tom Hanks’:20, ‘Denzel Washington’:15}
Your program should show the similarity score between the directors. (An example is given below).

**Run all the above block before you start with this question**

**Run the Next 2 box one after another to execute and check the output**

In [10]:
def director_cosine_similarity():                                                                              # function to calulate cosine similarity between two diretors
    print("Pick Any 2 Directors from the following list to calculate cosine similarity - \n")
    for single_director in director_appearences.keys():                                                        # printing all the directors names to select from
        print(single_director)
    print("\n")    
    director1 = input("Enter Director 1 Name-").strip().lower()                                                #taking input from user for director1
    director2 = input("Enter Director 2 Name-").strip().lower()                                                # taking input from user for director2
    while True:                                                                                                # iterating infinite times
        try:
            flag = 0                                                                                           # try block - setting flag to 0
            if director1 not in director_appearences.keys() and director2 not in director_appearences.keys():  # checking with both th user input are not there in directors list
                flag=3                                                                                         # if not present setting fkag to 3 and raising exception
                raise
            elif director1 not in director_appearences.keys() and director2 in director_appearences.keys():    # hceking if director1 not present and director2 present
                flag=1                                                                                         # setting flag to 1 and raising exception
                raise
            elif director1 in director_appearences.keys() and director2 not in director_appearences.keys():    #checking if director1 present and director2 not present
                flag=2                                                                                         # setting flag to 2 and raising exception
                raise
            else:                                                                                              # else both are present then break
                break
        except:                                                                                                #try block unable to execute completely
            if flag==3:                                                                                        # flag set to 3
                director1 = input("Enter Valid Director 1 Name-").strip().lower()                              #taking valid input from user for director1 and 2
                director2 = input("Enter Valid Director 2 Name-").strip().lower()
            elif flag==1: 
                director1 = input("Enter Valid Director 1 Name-").strip().lower()                              #if flag set to 1 and taking valid input from user for director 1 
            elif flag==2:
                director2 = input("Enter Valid Director 2 Name-").strip().lower()                              # if flag set to 2 and taking valid input from user for director 2
    
    director1_movies_cast = []                                                                                # empty director movie cast lists for director 1 and 2
    director2_movies_cast = []
    
    for i, information in enumerate(movies_full_info_list):                                                   # iterating through all movie information list and checking if the director is director 1 then appending all the cast and principle cast to list
        if director1 in information['director']:
            director1_movies_cast.extend(information['cast'])
            director1_movies_cast.extend(information['principle_cast'])
    
    for i, information in enumerate(movies_full_info_list):                                                    # iterating through all movie information list and checking if the director is director 1 then appending all the cast and principle cast to list
        if director2 in information['director']:
            director2_movies_cast.extend(information['cast'])
            director2_movies_cast.extend(information['principle_cast'])
    
    director1_actor_counts = {}                                                                            # empty dictionaries to store actor count for each director
    director2_actor_counts = {}
    
    for cast_member in director1_movies_cast:                                                              # iterating through each member in the cast and getting their counts and storing them in dictionary for appropriate director
        director1_actor_counts[cast_member] = director1_actor_counts.get(cast_member,0) + 1
    
    for cast_member in director2_movies_cast:                                                              # iterating through each member in the cast and getting their counts and storing them in dictionary for appropriate director
        director2_actor_counts[cast_member] = director2_actor_counts.get(cast_member,0) + 1
        
    combined_cast_set = (set(director1_actor_counts.keys())).union(set(director2_actor_counts.keys()))     # taking union of casts from both directors
    
    director1_vector = []                                                                                  # creating empty director vectors for each director
    director2_vector = []
    
    for combined_cast_member in combined_cast_set:                                                         # iterating through actor in the combined cast set and getting count values for each directors vetor items
        director1_vector.append(director1_actor_counts.get(combined_cast_member,0))
        director2_vector.append(director2_actor_counts.get(combined_cast_member,0))
    
    #calculate cosine similarity
    divident = 0                                                                                       # initializing divident to 0
    for i in range(len(director1_vector)):                                                             #iterating through the length of the director vectors                                              
        summing = director1_vector[i] * director2_vector[i]                                            # get temp sum from both directors vectors corresponding values
        divident = divident + summing                                                                  # adding temp sum to divident value for every iteration
        
    a = 0                                                                       #initializing to variables a and b to 0
    b = 0
    for i in range(len(director1_vector)):                                      #iterating through the length of directors vector
        sum1 = (director1_vector[i])**2                                         # temp sum with squared value of corresponding vector value
        a = a + sum1                                                            # adding temp squared sum to a
        sum2 = (director2_vector[i])**2                                         # temp sum with squared value of corresponding vector value
        b = b + sum2                                                            # adding temp squared sum to b
    divisor = (a**0.5)*(b**0.5)                                                 # divisor is assigned with the product of squared root of both a and b
    
    cosine_similarity = divident/divisor                                                                             # cosine similarity if calculated by divident by divisor
    print("Cosine Similarity between Directors " + director1 + " & " + director2 + " is "+ str(cosine_similarity))   # same is printed as message
     
    
    
    
    
    

In [11]:
director_cosine_similarity()

Pick Any 2 Directors from the following list to calculate cosine similarity - 

orson welles
francis ford coppola
alfred hitchcock
michael curtiz
richard linklater
krzysztof kieslowski
gene kelly
stanley donen
charles chaplin
barry jenkins
d.w. griffith
ben sharpsteen
bill roberts
hamilton luske
jack kinney
norman ferguson
t. hee
wilfred jackson
john huston
guillermo del toro
billy wilder
steve james
akira kurosawa
joseph l. mankiewicz
françois truffaut
sam peckinpah
jim sheridan
carol reed
stanley kubrick
jasmila zbanic
george cukor
sam wood
victor fleming
cristian mungiu
sergei m. eisenstein
elia kazan
george lucas
john elliotte
samuel armstrong
alfonso cuarón
ernst lubitsch
sidney lumet
kenneth lonergan
roman polanski
steve mcqueen
charles burnett
robert altman
brad bird
joon-ho bong
sarah gavron
nicolas roeg
john ford
marcel carné
martin scorsese
preston sturges
david hand
ford beebe jr.
james algar
jim handley
paul satterfield
hayao miyazaki
gary trousdale
kirk wise
john lasseter


Example: 

peter weir & michel gondry  - have Jim Carrey as one of the common  cast in their movies and have a cosine similarity of 0.066.

peter weir & peter weir himself to get a cosine similarity of 1.

peter weir & amir bar-lev - have no cast in common and have a cosine similarity of 0.

Cosine Similarity between Directors alfred hitchcock & orson welles is 0.031746031746031744

############################################################################################################################

3. Pick 5 of your favorite actors/actresses from this list of top 500 movies. Then create a dictionary of all the actors/actresses that they have collaborated with in a movie. Following similar method as above in task 2, create the dictionaries, and compare these 5 actors, who is the most popular supporting actor/actress among them all?

**Run all the above block before you start with this question**

**Run the Next 2 box one after another to execute and check the output**

In [12]:
def cosine_similarity_for_two(actor1, actor2):                            # function to cosine similarity between two actors
    actor1_movies_cast = []                                                                                 # empty list to store movie casts of boths actor1 and 2
    actor2_movies_cast = []
    
    for i, information in enumerate(movies_full_info_list):                                                 # iterating through all the movie information and checking the actor1 is found in the cast or principle cast and appending castand principle cast to appropriate lists
        if actor1 in information['cast'] or actor1 in information['principle_cast']:
            actor1_movies_cast.extend(information['cast'])
            actor1_movies_cast.extend(information['principle_cast'])
    
    for i, information in enumerate(movies_full_info_list):                                                 # iterating through all the movie information and checking the actor2 is found in the cast or principle cast and appending castand principle cast to appropriate lists
        if actor2 in information['cast'] or actor2 in information['principle_cast']:
            actor2_movies_cast.extend(information['cast'])
            actor2_movies_cast.extend(information['principle_cast'])
    
    actor1_cast_counts = {}                                                                               #creating two dictionaries to store actor counts
    actor2_cast_counts = {}
    
    for cast_member in actor1_movies_cast:                                                                #iterating through each actor casts and storing their counts to the appropriate dictionary
        if cast_member != actor1:
            actor1_cast_counts[cast_member] = actor1_cast_counts.get(cast_member,0) + 1
    
    for cast_member in actor2_movies_cast:                                                                #iterating through each actor casts and storing their counts to the appropriate dictionary
        if cast_member != actor2:
            actor2_cast_counts[cast_member] = actor2_cast_counts.get(cast_member,0) + 1
    
    actor1_vector = []                                                                                   # creating empty lists to vector values for each actor
    actor2_vector = []
    
    combined_cast_set = (set(actor1_cast_counts.keys())).union(set(actor2_cast_counts.keys()))           # creating union of casts from both the actors
    
    for combined_cast_member in combined_cast_set:                                                       # iterating through the combined cast from the 2 actors and stroing their count in respective dictionaries
        actor1_vector.append(actor1_cast_counts.get(combined_cast_member,0))
        actor2_vector.append(actor2_cast_counts.get(combined_cast_member,0))
    
#calculate cosine similarity
    divident = 0                                                                                       # initializing divident to 0
    for i in range(len(actor1_vector)):                                                             #iterating through the length of the director vectors                                              
        summing = actor1_vector[i] * actor2_vector[i]                                            # get temp sum from both directors vectors corresponding values
        divident = divident + summing                                                                  # adding temp sum to divident value for every iteration
        
    a = 0                                                                       #initializing to variables a and b to 0
    b = 0
    for i in range(len(actor1_vector)):                                      #iterating through the length of directors vector
        sum1 = (actor1_vector[i])**2                                         # temp sum with squared value of corresponding vector value
        a = a + sum1                                                            # adding temp squared sum to a
        sum2 = (actor2_vector[i])**2                                         # temp sum with squared value of corresponding vector value
        b = b + sum2                                                            # adding temp squared sum to b
    divisor = (a**0.5)*(b**0.5)                                                 # divisor is assigned with the product of squared root of both a and b
    
    cosine_similarity = divident/divisor                                        # dividing divident by divisor and storing it to cosine_similarity
    return cosine_similarity                                                    # returnin

In [13]:
five_actors_list = ['amy adams', 'scarlett johansson', 'chris pratt', 'joaquin phoenix', 'adam driver' ]  # five selected actors/actresses

all_cast_members_for_actor_list = []                                                  # to store all the cast members where these actors are found                                            
for actor in five_actors_list:                                                        # iterating through each actor in the 5 actors list
    for i, information in enumerate(movies_full_info_list):                           # iterating through all movies information   
        if actor in information['cast'] or actor in information['principle_cast']:    # checking if the actor is in the cast or principle cast of every
            all_cast_members_for_actor_list.extend(information['cast'])               # appending all cast and principle cast to cast members list
            all_cast_members_for_actor_list.extend(information['principle_cast'])

supporting_cast_count_for_actor = {}                                                                           # supporting cast count dictionary

for cast_member in all_cast_members_for_actor_list:                                                               # iterating through all the cast members of the for actors
    if cast_member not in five_actors_list:                                                                       # checking if cast is not in five actors selected
        supporting_cast_count_for_actor[cast_member] = supporting_cast_count_for_actor.get(cast_member,0) + 1     # adding the cast member count to support cast members count dictionary
        
most_popular_supporting_cast = []                                                    # list to store most popular supporting actor
maximum = max(supporting_cast_count_for_actor.values())                              # storing maximum count in the supporting actors count
for cast_member, count in supporting_cast_count_for_actor.items():                   # iterating through each supporting actor in the list
    if count == maximum:                                                             # checking if the count of a specific actor is equal to maximum
        most_popular_supporting_cast.append(cast_member)                             # adding cast member to most popular supporting actor list
        
cosine_similarity_matrix = [[0 for i in range(5)] for i in range(5)]                 # creating a 5x5 matrix
        
for i in range(len(five_actors_list)):                                               # iterating through each actor in the 5 actors
    for j in range(len(five_actors_list)):                                           # iterating through each actor in 5 actors
        if i!=j:                                                                     # when its not the same actor
            cosine_similarity_matrix[i][j] = cosine_similarity_for_two(five_actors_list[i], five_actors_list[j])          # calling cosine similarity function and passing the ith and jth actor as parameter to it
        else:
            cosine_similarity_matrix[i][j] = 1.0                                       # if it is the same actor then store value 1 in the cosine similarity matrix
            
print("There are", len(most_popular_supporting_cast), "Most Famous supporting Actor/Actresses in the combined cast of the 5 Actors -\n")          
for i in range(len(most_popular_supporting_cast)):                                       # printing each cast member in the most famous supporting actors list
    print(most_popular_supporting_cast[i])
            
print("\nCosine Similarity between the 5 actors are - \n")
for i in range(5):
    for j in range(i+1,5):
        print("{:<20} {:<20} {:<10}".format(five_actors_list[i] , five_actors_list[j], cosine_similarity_matrix[i][j]))                #printing both actors in comparison  and the cosine similarity among them from the matrix                                        




There are 3 Most Famous supporting Actor/Actresses in the combined cast of the 5 Actors -

olivia wilde
rooney mara
samantha morton

Cosine Similarity between the 5 actors are - 

amy adams            scarlett johansson   0.44317413485081447
amy adams            chris pratt          0.5123145973175687
amy adams            joaquin phoenix      0.7055358336498685
amy adams            adam driver          0.01641385684632646
scarlett johansson   chris pratt          0.4246574522630248
scarlett johansson   joaquin phoenix      0.5848184907608683
scarlett johansson   adam driver          0.40816326530612246
chris pratt          joaquin phoenix      0.6760571658787814
chris pratt          adam driver          0.015728053787519437
joaquin phoenix      adam driver          0.021659944102254382


############################################################################################################################